In [1]:
%load_ext autoreload
%autoreload 2

# In-silico color mix experiment

This notebook walks through how to run an 'in-silico' color mixing experiment. This uses a simulated/calculated color mixing calculation to create and measure mixed colors instead of mixing and measuring them on Jubilee. This makes it easier and faster to experiment with the optimization details without getting stuck on Jubilee motion platform problems

## Launch the bayesian optimization server 

This implementation uses my bo-serving package to serve a bayesian optimization service as an HTTP API endpoint. Everything to do this is configured within the color mixing code we use here. All that needs to be done is to start up the service:

1. Clone the bo-serving repo: https://github.com/pozzo-research-group/bo-serving
2. Install the bo-serving repo: from the root of the repo, run `pip install -e .`
3. Change directories into 'src/bo_serving/botorch_serving'
4. Run the command `flask --app serve_botorch run` to launch the server

If step 4 ran correctly, you are all set!

In [9]:
import cv2
import numpy as np
from PIL import Image

from jubilee_pipette_bodemo.color_matcher import ColorMatcher
from jubilee_pipette_bodemo.in_silico_mixing import synthetic_image
import jubilee_pipette_bodemo.image_processing as image_processing

import requests

# ------ Data visualization outside of the notebook ------
import matplotlib
#matplotlib.use('Qt5Agg')

import io

import torch

## Create the color matcher object

This creates the 'ColorMatcher' object with the parameters we want to use for testing. Here, we are using 5 total stocks, a 'sample volume' of 300 (this is a carryover from the experimental color mixing and lets us use the same python code for both versions), a euclidean distance color score, simulated mixing, and the 5 colors in the 'in silico colors' list are the colors our experiment will use to mix.

In [10]:
url = 'http://127.0.0.1:5000'
n_params = 5
n_bo_trials = 20
n_random_trials = 5
init_response = requests.post(url + '/new_experiment', json = {'n_params':n_params, 'n_random_trials':n_random_trials, 'n_bo_trials':n_bo_trials})

In [4]:
init_response

<Response [200]>

In [11]:
RobRoss = ColorMatcher(5, sample_volume=300, score_type='euclidean', in_silico_mixing = True, in_silico_colors = [[255, 0, 0], [255, 255, 0], [0,0,255], [255,255,255], [0,0,0]], n_random_its=5, n_bo_its=20, http_url='http://127.0.0.1:5000')

## Pick a color

You need to click on the blue box and select a target color for the selection to register correctly as an RGB value

In [12]:
target_color = RobRoss.select_target_color()

ColorPicker(value='blue', description='Pick a color')

## Run the campaign

This will run the experiment. The first number will determine the number of total experiments run. The first n_random_its experiments (specified when we created the RobRoss object) will be run with random sample selection, and the rest will use bayesian optimization.

In [ ]:
RobRoss.run_campaign(25, None, None, None, None, None, starting_well = 0, save =True)

http optimizer is asking service for next trial experiment 298264388507862907309965493277379901365
new trial index:  0
query point:  [0.23456798213205282, 0.019112375083100368, 0.3258069886033031, 0.32853841758338037, 0.09197423659816337]
type query pt:  <class 'list'>
RYB values tested: [0.23456798213205282, 0.019112375083100368, 0.3258069886033031, 0.32853841758338037, 0.09197423659816337]
RGB values observed: (140, 73, 155)
color score:  [0.9574813172563946]
type:  <class 'list'>
Updating optimizer for experiment 298264388507862907309965493277379901365
http optimizer is asking service for next trial experiment 298264388507862907309965493277379901365
new trial index:  1
query point:  [0.3279738530744181, 0.04577686041110876, 0.004651129953449276, 0.3599419385654521, 0.2616562179955717]
type query pt:  <class 'list'>
RYB values tested: [0.3279738530744181, 0.04577686041110876, 0.004651129953449276, 0.3599419385654521, 0.2616562179955717]
RGB values observed: (192, 87, 85)
color score:

## Open the dashboard to view performance:

1. From the root of the Jubilee_pipette_BOdemo repo, change directories into 'streamlit_dash'
2. Run 